# Imbalanced classification: credit card fraud detection. RC edit

**Author:** [fchollet](https://twitter.com/fchollet)<br>
**Date created:** 2019/05/28<br>
**Last modified:** 2020/04/17<br>
**Description:** Demonstration of how to handle highly imbalanced classification problems -> use class_weight in fit

https://keras.io/examples/structured_data/imbalanced_classification/

## Introduction

This example looks at the
[Kaggle Credit Card Fraud Detection](https://www.kaggle.com/mlg-ulb/creditcardfraud/)
dataset to demonstrate how
to train a classification model on data with highly imbalanced classes.

## First, vectorize the CSV data

In [1]:
ls

creditcard.csv  sample_data/


In [2]:
import csv
import numpy as np

# Get the real data from https://www.kaggle.com/mlg-ulb/creditcardfraud/
fname = "creditcard.csv"

all_features = []
all_targets = []
with open(fname) as f:
    for i, line in enumerate(f):
        if i == 0:
            print("HEADER:", line.strip())
            continue  # Skip header
        fields = line.strip().split(",")
        all_features.append([float(v.replace('"', "")) for v in fields[:-1]])
        all_targets.append([int(fields[-1].replace('"', ""))])
        if i == 1:
            print("EXAMPLE FEATURES:", all_features[-1])

HEADER: "Time","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount","Class"
EXAMPLE FEATURES: [0.0, -1.3598071336738, -0.0727811733098497, 2.53634673796914, 1.37815522427443, -0.338320769942518, 0.462387777762292, 0.239598554061257, 0.0986979012610507, 0.363786969611213, 0.0907941719789316, -0.551599533260813, -0.617800855762348, -0.991389847235408, -0.311169353699879, 1.46817697209427, -0.470400525259478, 0.207971241929242, 0.0257905801985591, 0.403992960255733, 0.251412098239705, -0.018306777944153, 0.277837575558899, -0.110473910188767, 0.0669280749146731, 0.128539358273528, -0.189114843888824, 0.133558376740387, -0.0210530534538215, 149.62]


In [3]:
features = np.array(all_features, dtype="float32")
targets = np.array(all_targets, dtype="uint8")
print("features.shape:", features.shape)
print("targets.shape:", targets.shape)

features.shape: (284807, 30)
targets.shape: (284807, 1)


## Prepare a validation set

In [8]:
num_val_samples = int(len(features) * 0.2)
train_features = features[:-num_val_samples]
train_targets = targets[:-num_val_samples]
val_features = features[-num_val_samples:]
val_targets = targets[-num_val_samples:]

print("Number of training samples:", len(train_features))
print("Number of validation samples:", len(val_features))


Number of training samples: 227846
Number of validation samples: 56961


In [9]:
val_targets.sum()

75

In [19]:
100 * val_targets.sum() / len(val_targets)

0.13166903670932745

75 fraudulant transactions in the val set, about 0.13%

## Analyze class imbalance in the targets

In [10]:
counts = np.bincount(train_targets[:, 0])
print(
    "Number of positive samples in training data: {} ({:.2f}% of total)".format(
        counts[1], 100 * float(counts[1]) / len(train_targets)
    )
)

weight_for_0 = 1.0 / counts[0]
weight_for_1 = 1.0 / counts[1]


Number of positive samples in training data: 417 (0.18% of total)


Pretty close % to validation set

## Normalize the data using training set statistics
Subtract mean and divide by SD

In [11]:
mean = np.mean(train_features, axis=0)
train_features -= mean
val_features -= mean
std = np.std(train_features, axis=0)
train_features /= std
val_features /= std


## Build a binary classification model
https://keras.io/guides/sequential_model/

A Sequential model is appropriate for a plain stack of layers where each layer has exactly one input tensor and one output tensor.

In [12]:
from tensorflow import keras

model = keras.Sequential(
    [
        keras.layers.Dense(
            256, activation="relu", input_shape=(train_features.shape[-1],)
        ),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               7936      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 139,777
Trainable params: 139,777
Non-trainable params: 0
__________________________________________________

## Train the model with `class_weight` argument

In [13]:
metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-2), loss="binary_crossentropy", metrics=metrics
)

callbacks = [keras.callbacks.ModelCheckpoint("fraud_model_at_epoch_{epoch}.h5")]
class_weight = {0: weight_for_0, 1: weight_for_1}

model.fit(
    train_features,
    train_targets,
    batch_size=2048,
    epochs=30,
    verbose=2,
    callbacks=callbacks,
    validation_data=(val_features, val_targets),
    class_weight=class_weight,
)


Epoch 1/30
112/112 - 1s - loss: 2.3824e-06 - fn: 49.0000 - fp: 24872.0000 - tn: 202557.0000 - tp: 368.0000 - precision: 0.0146 - recall: 0.8825 - val_loss: 0.1398 - val_fn: 8.0000 - val_fp: 1899.0000 - val_tn: 54987.0000 - val_tp: 67.0000 - val_precision: 0.0341 - val_recall: 0.8933
Epoch 2/30
112/112 - 1s - loss: 1.4327e-06 - fn: 34.0000 - fp: 6955.0000 - tn: 220474.0000 - tp: 383.0000 - precision: 0.0522 - recall: 0.9185 - val_loss: 0.1223 - val_fn: 6.0000 - val_fp: 2242.0000 - val_tn: 54644.0000 - val_tp: 69.0000 - val_precision: 0.0299 - val_recall: 0.9200
Epoch 3/30
112/112 - 1s - loss: 1.4953e-06 - fn: 30.0000 - fp: 8791.0000 - tn: 218638.0000 - tp: 387.0000 - precision: 0.0422 - recall: 0.9281 - val_loss: 0.0386 - val_fn: 10.0000 - val_fp: 363.0000 - val_tn: 56523.0000 - val_tp: 65.0000 - val_precision: 0.1519 - val_recall: 0.8667
Epoch 4/30
112/112 - 1s - loss: 1.4597e-06 - fn: 27.0000 - fp: 6234.0000 - tn: 221195.0000 - tp: 390.0000 - precision: 0.0589 - recall: 0.9353 - val_l

In [14]:
predictions = model.predict(val_features)

In [18]:
len(val_features)

56961

In [23]:
def calc_precision(tp, fp):
  return tp / (tp + fp)

def calc_recall(tp, fn):
  return tp / (tp + fn)

print(calc_precision(tp=402, fp=5186))

print(calc_recall(tp=402, fn=15))

0.07193987115246958
0.9640287769784173


Final step
112/112 - 1s - 
loss: 4.2126e-06 - 

* fn: 15.0000
* fp: 5186.0000
* tn: 222243.0000
* tp: 402.0000
* precision: 0.0719
* recall: 0.9640


* val_loss: 0.0572
* val_fn: 10.0000 - Missing 10 fraudulent transactions
* val_fp: 850.0000 - incorrectly flagging 850 legitimate transactions
* val_tn: 56036.0000
* val_tp: 65.0000 - Correctly identifying 65 of them as fraudulent
* val_precision: 0.0710
* val_recall: 0.8667

## Conclusions

At the end of training, out of 56,961 validation transactions, we are:

- Correctly identifying 65 of them as fraudulent
- Missing 10 fraudulent transactions
- At the cost of incorrectly flagging 850 legitimate transactions

In the real world, one would put an even higher weight on class 1,
so as to reflect that False Negatives are more costly than False Positives.

Next time your credit card gets  declined in an online purchase -- this is why.